In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-i7wzt6y0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-i7wzt6y0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-nvqztvri/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [4]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmp7kwtsano".


**Question1 (Q1)**

In [7]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(float *arr, int size,float *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ float partialSum[];

unsigned int thread_id = blockIdx.x * blockDim.x + threadIdx.x ;

unsigned int t = threadIdx.x;

partialSum[t] = arr[thread_id];

if(thread_id >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 float* array = nullptr;  // Pointer to the array
 float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     float *array_device;
     float *result_device;

     result = (float*)malloc(sizeof(float));
     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(256, 1);
    dim3 numBlocks(1,1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"final sum is: "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Writing Q1_2_31_2_33.cu


In [49]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1
!./Q1 "input.txt"

1.2
3.8
6.4
7.9
final sum is: 19.3


**Question2 (Q2)**

In [24]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void BinarySearch(float *array, int size,float target,float *result)
{

    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    // Calculate the segment size for each thread
    int segment_size = (size + blockDim.x - 1) / blockDim.x;
    int start = tid * segment_size;
    int end = min(start + segment_size, size) - 1;

    // Binary search within the assigned segment
    int res = -1;

    while (start <= end) {
        int mid = (start + end) / 2;

        if (array[mid] == target) {
            atomicMax((int*)result, mid);
            break;
        } else if (array[mid] < target) {
            start = mid + 1;
        } else {
            end = mid - 1;
        }
    }

    // Synchronize threads before returning
    __syncthreads();
    if (threadIdx.x == 0) {
        *result = res;
    }


}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);
    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 float* array = nullptr;  // Pointer to the array
 float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     float *array_device;
     float *result_device;

     result = (float*)malloc(sizeof(float));
     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(256, 1);
    dim3 numBlocks(1,1);
    BinarySearch<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count, target,result_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"final sum is: "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q2_2_31_2_33.cu


In [25]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2
!./Q2 "input.txt" 6.4

1.2
3.8
6.4
7.9
final sum is: -1
